### Start the kernel specific to SPARK

* conda env list
* conda activate spark
* in visual code IDE, select the environment as python-spark
* check that in the selected environment pyspark is installed

In [ ]:
!pip3 show pyspark
!pip3 show findspark

In [ ]:
# The following line help the Jupyter program to find the Spark binaries to run the job
import findspark
findspark.init()

In [ ]:
# Sample program to validate pySpark library is available
import pyspark
sc = pyspark.SparkContext('local[*]')

txt = sc.textFile('file:////Users/sxxx/github/spark-scala/README.md')
print(txt.count())

python_lines = txt.filter(lambda line: 'python' in line.lower())
print(python_lines.count())


#big_list = range(10000)
#>>> rdd = sc.parallelize(big_list, 2)
#>>> odds = rdd.filter(lambda x: x % 2 != 0)
#>>> odds.take(5)

In [ ]:
# Another random code snippet to check if the Spark session is still alive after the previous cell execution
big_list = range(10000)
rdd = sc.parallelize(big_list, 2)
odds = rdd.filter(lambda x: x % 2 != 0)
odds.take(5)

In [ ]:
# Load the CCM properties file
from os.path import expanduser
home = expanduser("~")

separator = "="
keys = {}

# I named your file conf and stored it 
# in the same directory as the script

with open(home+'/nexus.prop') as f:

    for line in f:
        if separator in line:

            # Find the name and value by splitting the string
            name, value = line.split(separator, 1)

            # Assign key value pair to dict
            # strip() removes white space from the ends of strings
            keys[name.strip()] = value.strip()

#print(keys)

In [ ]:
#This section handles adding JDBC driver to the PYSPARK shell
import os
jdbc_connector_mysql=keys["jdbc-connector-mysql"]

os.environ["PYSPARK_SUBMIT_ARGS"] = f"--jars file://{jdbc_connector_mysql} pyspark-shell"


In [ ]:
#Create sparksession instance

spark.stop()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Fetch 1 workflow records from the templates table
#database = 'reach'
#url='jdbc:mysql://{}/{}'.format(host, database)
#table = 'reach.tasks'

table = "(SELECT workflowId, REPLACE(CONVERT(REPLACE(workflow,'&amp;','&') using utf8),'','') as workflow FROM reach.templates LIMIT 3) AS t"

url = keys["reach-dev-url"]
#query='select taskid from tasks limit 10'
user = keys["reach-dev-userid"]
password = keys["reach-dev-password"]

properties = {
    'user': user,
    'password': password,
    'driver': 'com.mysql.jdbc.Driver',
    #'query': 'select taskid from tasks limit 10',
    'fetchsize': '10'
}

df = spark.read.jdbc(url, table, properties=properties)
#df = sqlContext.read.jdbc(url, query, properties=properties)
df.show()

In [ ]:
#Parse the json column
df.select("workflow").show()